In [13]:
import pandas as pd


def get_data():

    file_path_train = "artificial_train.data"

    df_train = pd.read_csv(file_path_train, sep=" ", header=None)
    df_train = df_train.drop(df_train.columns[-1], axis=1)

    # print(df_train)

    X_train = df_train

    file_path_test = "artificial_test.data"

    df_test = pd.read_csv(file_path_test, sep=" ", header=None)
    df_test = df_test.drop(df_test.columns[-1], axis=1)

    # print(df_test)

    X_test = df_test

    file_path_train_labels = "artificial_train.labels"

    df_train_labels = pd.read_csv(file_path_train_labels, sep=" ", header=None)

    # print(df_train_labels)

    y_train = df_train_labels

    return X_train, y_train, X_test

X_train, y_train, X_test = get_data()

print(f"Done downloading")

Done downloading


In [ ]:
%pip install autogluon

In [17]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

# Load data
X_train_raw, y_train_raw, X_test_raw = get_data()

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_raw, y_train_raw, test_size=0.2, random_state=54)

# Combine X_train and y_train for AutoGluon
train_data = X_train.copy()
train_data['label'] = y_train

# Initialize and train the AutoGluon predictor
predictor = TabularPredictor(label='label', eval_metric='balanced_accuracy').fit(train_data, presets='best_quality', time_limit=200)

No path specified. Models will be saved in: "AutogluonModels\ag-20240112_084009"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240112_084009/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.


In [16]:
# Making predictions on the validation set
y_val_pred = predictor.predict(X_val)

# Calculating balanced accuracy
balanced_accuracy_val = balanced_accuracy_score(y_val, y_val_pred)
print(f"Balanced Accuracy on Validation Set: {balanced_accuracy_val}")

# Making predictions on the test set (X_test_raw)
test_predictions = predictor.predict(X_test_raw)

# If you need probabilities for each class
test_probabilities = predictor.predict_proba(X_test_raw)

# Writing probabilities to a file
with open('predictions.txt', 'w') as file:
    for prob in test_probabilities.iloc[:, 1]:
        file.write(f"{prob}\n")

print("Predictions and probabilities written to 'predictions.txt'")

ValueError: Found input variables with inconsistent numbers of samples: [2000, 600]